In [1]:
#Import libraries
#!pip install kaggle
#import kaggle

In [2]:
#Download dataset from kaggle api
#!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [5]:
#orders.csv contain "PK\x03\x04" as first line, which is a magic header for zip file, so it is zipped archive
#To obtain the csv file, unzip first but in read mode 'r' and not extractall() as file in not .zip
#Unzip and read data from the file and handle null values
import pandas as pd
import zipfile
zip_path = "orders.csv"
with zipfile.ZipFile(zip_path, 'r') as z:
    print(z.namelist())
    with z.open(zip_path) as f:
        df = pd.read_csv(f, na_values = ['Not Available', 'unknown'])


['orders.csv']


In [7]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [12]:
#rename column names...make them lowercase and replace space with underscore
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [14]:
#Create new features, discount, sale_price and profit
df["discount"] = df["list_price"] * df["discount_percent"] * 0.01
df["sale_price"] = df["list_price"] - df["discount"]
df["profit"] = df["sale_price"] - df["cost_price"]
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [18]:
#correct data type of feature "order_date" from "object" to "Date"
df["order_date"] = pd.to_datetime(df["order_date"], format = "%Y-%m-%d")

In [19]:
df = df.drop(columns = ["list_price", "cost_price", "discount_percent"])

In [20]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [28]:
#Create SQL server connection using sqlalchemy
import urllib.parse
from sqlalchemy import create_engine
username = "your_username"
password = "your_password"
server = r"your_servername"   
database = "your_database"
driver = "ODBC Driver 18 for SQL Server"

# Build an ODBC connection string and URL-encode it
odbc_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes"   # Driver 18 encrypts by default
)
params = urllib.parse.quote_plus(odbc_str)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
with engine.connect() as conn:
    print(conn.exec_driver_sql("SELECT @@VERSION").scalar())
conn = engine.connect()

Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Express Edition (64-bit) on Windows 10 Home Single Language 10.0 <X64> (Build 19045: )



In [29]:
#Load the data into sql server using append
df.to_sql("df_orders", con=conn, index=False, if_exists="append")

38

In [32]:
df.shape

(9994, 16)